# 1. Importations et initialisations nécessaires

In [1]:
import zipfile
import pandas as pd
import gensim, logging
from tqdm.notebook import tqdm
from utils.read_data import getDF
from IPython.core.display import HTML
from utils.useful_functions import PATH_DATA
from utils.useful_functions import sim_cosine
from utils.useful_functions import PATH_METADATA
from utils.useful_functions import clear_description
from utils.useful_functions import top_n_recommendation
from utils.useful_functions import imUrl_to_image_html_width100

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO) # Configuration de la journalisation 

# 2. Récupération et visualisation des données

In [2]:
# Décompression du fichier dataset.zip
with zipfile.ZipFile(PATH_DATA, 'r') as data:
    data.extractall()

In [3]:
# Lecture des données du fichier metadata.json
data = getDF(PATH_METADATA)

In [4]:
# Visualisation des 5 premières lignes du DataFrame data
display(
    data.head(5).style.format("{0}").set_caption("5 premières lignes du DataFrame data après chargement des données : Dimension = " +
                                                          str(data.shape)
                                                         )
)

,ID,title,slug,category,imPath
0,7541,Christina Gavioli,christina-gavioli-3,"['Fashion Women', 'Women Blouse and Dress']",images/Fashion Women/Women Blouse and Dress/CHRISTINA_GAVIOLI.jpg
1,7540,Sexy Woman,sexy-woman-3,"['Fashion Women', 'Women Blouse and Dress']",images/Fashion Women/Women Blouse and Dress/SEXY_WOMAN_MULTICOLORE.jpg
2,7539,Sexy Woman,sexy-woman-2,"['Fashion Women', 'Women Blouse and Dress']",images/Fashion Women/Women Blouse and Dress/SEXY_WOMAN_JAUNE.jpg
3,7538,Christina Gavioli,christina-gavioli-2,"['Fashion Women', 'Women Blouse and Dress']",images/Fashion Women/Women Blouse and Dress/christina_gavioli_bordeau.jpg
4,7537,Christina Gavioli,christina-gavioli,"['Fashion Women', 'Women Blouse and Dress']",images/Fashion Women/Women Blouse and Dress/christina_gavioli_blc_maron.jpg


# 3. Prétraitement et visualisation des données

In [5]:
# Supression des colonnes: slug et category du Dataframe data
del data['slug']
del data['category']

In [6]:
# Localisation horizontale des valeurs manquantes dans le DataFrame data
pd.isnull(data).any(axis=0)

ID        False
title     False
imPath    False
dtype: bool

In [7]:
# Les titres des produits sont utilisés comme descriptions des produits
# Prétraitement des descriptions des produits, nous entendons par là :
# - Conversion des descriptions en minuscule
# - Suppression des caractères de ponctuation
# - Tokénisation
# - Suppression des mots vides
data['tokens_list'] = data['title'].apply(lambda x : clear_description(x))

In [8]:
# Visualisation des 5 premières lignes du DataFrame data
display(
    data.head(5).style.format("{0}").set_caption("5 premières lignes du DataFrame data prétraité : Dimension = " +
                                                          str(data.shape)
                                                         )
)

,ID,title,imPath,tokens_list
0,7541,Christina Gavioli,images/Fashion Women/Women Blouse and Dress/CHRISTINA_GAVIOLI.jpg,"['christina', 'gavioli']"
1,7540,Sexy Woman,images/Fashion Women/Women Blouse and Dress/SEXY_WOMAN_MULTICOLORE.jpg,"['sexy', 'woman']"
2,7539,Sexy Woman,images/Fashion Women/Women Blouse and Dress/SEXY_WOMAN_JAUNE.jpg,"['sexy', 'woman']"
3,7538,Christina Gavioli,images/Fashion Women/Women Blouse and Dress/christina_gavioli_bordeau.jpg,"['christina', 'gavioli']"
4,7537,Christina Gavioli,images/Fashion Women/Women Blouse and Dress/christina_gavioli_blc_maron.jpg,"['christina', 'gavioli']"


# 4. Apprentissage des words embeddings via le modèle Skip-Gram

In [9]:
# Initialisation et entrainement du modèle
sentences = list(data['tokens_list'])
model = gensim.models.Word2Vec(sentences=sentences, vector_size=300, window=5, min_count=1, workers=4, sg=1, epochs=10)

2022-04-03 11:37:32,937 : INFO : collecting all words and their counts
2022-04-03 11:37:32,953 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-04-03 11:37:32,965 : INFO : collected 1468 word types from a corpus of 7758 raw words and 1655 sentences
2022-04-03 11:37:32,968 : INFO : Creating a fresh vocabulary
2022-04-03 11:37:33,003 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 retains 1468 unique words (100.0%% of original 1468, drops 0)', 'datetime': '2022-04-03T11:37:33.003605', 'gensim': '4.1.2', 'python': '3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19043-SP0', 'event': 'prepare_vocab'}
2022-04-03 11:37:33,007 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 leaves 7758 word corpus (100.0%% of original 7758, drops 0)', 'datetime': '2022-04-03T11:37:33.007605', 'gensim': '4.1.2', 'python': '3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]', 'platf

In [10]:
# Suppression du champs tokens_list
del data['tokens_list']

# 5. Top-N-Recommendation

## 5.1. Configurations utiles

### 5.1.1. Taille de la liste de recommandation

In [11]:
N = 10

### 5.1.2. Produit concerné par la recommandation

In [12]:
# Liste des index des produits
print("Il y'a en tout :", len(data.index),"produits qui sont indexés de 0 à", len(data.index) - 1 )

Il y'a en tout : 1655 produits qui sont indexés de 0 à 1654


In [13]:
i = 798 # Faites varier i pour changer de produit 

## 5.2. Visualisation du produit choisi

In [14]:
# Création du DataFrame nécessaire pour la visualisation
asin = data.iloc[i]['ID'] # Identifiant du produit choisi
item_df = data[data['ID'] == asin]
item_df = pd.DataFrame(item_df)
item_df.rename(columns={'imPath' : 'image'}, inplace=True)

In [15]:
 # Rendu de l'images dans le DataFrame item_df
HTML(item_df.to_html(escape=False, formatters=dict(image=imUrl_to_image_html_width100)))

,ID,title,image
798,5710,Robe mini caba en coton et tissus,


## 5.3. Visualisation des produits recommandés à l'utilisateur

In [ ]:
# Détermination des produits à recommander à l'utisateur
recommended_products = top_n_recommendation(asin, N, model, data, sim_cosine)

In [17]:
# Fabrication du DataFrame qui présentera la liste de recommandation
similarity_score = []
asin = []
for i in range(len(recommended_products)):
    similarity_score.append(recommended_products[i][0])
    asin.append(recommended_products[i][1])

recommended_products = pd.DataFrame({"ID" : asin, "cosine_similarity" : similarity_score})
recommended_products = pd.merge(recommended_products, data, how='inner')
recommended_products.rename(columns={'imPath' : 'image'}, inplace=True)

In [18]:
# Rendu des imagaes dans le DataFrame recommended_products
HTML(recommended_products.to_html(escape=False, formatters=dict(image=imUrl_to_image_html_width100)))

,ID,cosine_similarity,title,image
0,5667,0.999911,Robe mini caba carrelé marié de tissus coton glacé,
1,5823,0.999899,Mini robe jaune collection K.Lessman en coton lourd,
2,5834,0.999890,Mini robe PLB évasée en coton couleur bleue avec perles,
3,5835,0.999887,Mini robe évasée coupe princesse PLB couleur rose en coton,
4,5847,0.999874,"Mini robe dentelle rose, démembré avec un col arrondi",
5,5861,0.999874,"Mini Robe Dame noire fleurie, 100¨% coton dur démembré.",
6,5860,0.999868,"Mini Robe Mousseline sexy, couleur blanche avec des motifs imprimés.",
7,5665,0.999866,Robe mini Caba afritude femme fleurie marié de coton jaune,
8,5808,0.999860,Mini robe panthère or en coton et à l'élasthanne,
9,5666,0.999858,Robe mini caba afritude Panthère noir et rose marié de coton,
